## <b>1 <span style='color:#4682B4'>|</span> Загрузка данных </b>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from catboost import CatBoostClassifier
import optuna

In [2]:
train_df = pd.read_parquet("/kaggle/input/alfa-x-finu-hack/train_data.pqt")
test_df = pd.read_parquet("/kaggle/input/alfa-x-finu-hack/test_data.pqt")
train_df.head()

,id,date,balance_amt_avg,balance_amt_max,balance_amt_min,balance_amt_day_avg,channel_code,city,city_type,index_city_code,...,cnt_cred_g_oper_3m,cnt_days_cred_g_oper_3m,sum_deb_h_oper_3m,cnt_deb_h_oper_3m,cnt_days_deb_h_oper_3m,sum_cred_h_oper_3m,cnt_cred_h_oper_3m,cnt_days_cred_h_oper_3m,start_cluster,end_cluster
0,0,month_1,0.744845,0.705492,1.287207,0.748101,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.951166,0.568681,0.897565,0.553624,0.774354,0.936506,0.295984,0.967947,"{α, γ}",{other}
1,0,month_2,1.049605,0.831916,2.458609,1.053805,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.948812,0.499716,0.785029,0.551904,0.696576,0.990157,0.298873,0.945969,"{α, γ}",{other}
2,0,month_3,0.692653,0.740253,0.430042,0.695747,channel_code_5,city_23,city_type_0,index_city_code_39,...,0.946458,0.442244,0.877050,0.551044,0.663243,0.810065,0.294829,0.956958,"{α, γ}",{other}
3,1,month_1,-0.081586,-0.091860,-0.114040,-0.080890,channel_code_2,city_14,city_type_0,None,...,0.945281,0.407762,0.369318,0.567093,0.785465,-0.184002,0.253523,0.462452,{other},{other}
4,1,month_2,-0.094962,-0.100504,-0.119302,-0.094307,channel_code_2,city_14,city_type_0,None,...,0.946066,0.430750,0.067275,0.559928,0.696576,-0.183854,0.255545,0.495419,{other},{other}


## <b>2 <span style='color:#4682B4'>|</span> Предобработка данных </b>

In [3]:
# Список категориальных признаков
cat_features = [
    "channel_code", "city", "city_type",
    "okved", "segment", "start_cluster",
    "index_city_code", "ogrn_month", "ogrn_year"
]

# Преобразование категориальных признаков в тип 'category'
for feature in cat_features:
    train_df[feature] = train_df[feature].astype("category")
    test_df[feature] = test_df[feature].astype("category")

# Подготовка данных для обучения модели
X_train = train_df.drop(["id", "date", "end_cluster"], axis=1)
y_train = train_df["end_cluster"]

# Разделение данных на обучающий и валидационный наборы
x_train, x_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Определение числовых столбцов после заполнения пропущенных значений
numerical_cols = x_train.select_dtypes(include='number').columns
    
# Используем SimpleImputer для заполнения пропущенных значений в числовых признаках средним
mean_imputer = SimpleImputer(strategy='mean') 
x_train[numerical_cols] = mean_imputer.fit_transform(x_train[numerical_cols])
x_val[numerical_cols] = mean_imputer.transform(x_val[numerical_cols])
test_df[numerical_cols] = mean_imputer.transform(test_df[numerical_cols])

# Используем SimpleImputer для заполнения пропущенных значений в категориальных признаках
simple_imputer = SimpleImputer(strategy='most_frequent') 
x_train[cat_features] = simple_imputer.fit_transform(x_train[cat_features])
x_val[cat_features] = simple_imputer.transform(x_val[cat_features])
test_df[cat_features] = simple_imputer.transform(test_df[cat_features])

# Инициализация RobustScaler
scaler = RobustScaler()

# Применение RobustScaler к числовым переменным
x_train[numerical_cols] = scaler.fit_transform(x_train[numerical_cols])
x_val[numerical_cols] = scaler.transform(x_val[numerical_cols])
test_df[numerical_cols] = scaler.transform(test_df[numerical_cols])

In [4]:
def weighted_roc_auc(y_true, y_pred, labels, weights_dict):
    unnorm_weights = np.array([weights_dict[label] for label in labels])
    weights = unnorm_weights / unnorm_weights.sum()
    classes_roc_auc = roc_auc_score(y_true, y_pred, labels=labels,
                                    multi_class="ovr", average=None)
    return sum(weights * classes_roc_auc)

cluster_weights = pd.read_excel("/kaggle/input/alfa-x-finu-hack/cluster_weights.xlsx").set_index("cluster")
weights_dict = cluster_weights["unnorm_weight"].to_dict()

## <b>3 <span style='color:#4682B4'>|</span> Подбор гиперпараметров при помощи Optuna </b>

In [5]:
def objective(trial):
    params = {
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'random_strength': trial.suggest_float('random_strength', 0.0, 100.0),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 20),
    }

    model = CatBoostClassifier(iterations=5_000,
                               loss_function='MultiClass',
                               task_type='GPU',
                               **params)

    model.fit(x_train,
              y_train,
              cat_features=cat_features,
              eval_set=(x_val, y_val),
              verbose=5000,
              early_stopping_rounds=100)

    y_pred_proba = model.predict_proba(x_val)
    score = weighted_roc_auc(y_val, y_pred_proba, model.classes_, weights_dict)

    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=40) 

[I 2024-04-21 22:12:46,355] A new study created in memory with name: no-name-685c0ce5-1717-47ba-a65d-1aad31b08bbd


0:	learn: 1.5229880	test: 1.5256852	best: 1.5256852 (0)	total: 320ms	remaining: 26m 40s
bestTest = 0.643606901
bestIteration = 4225
Shrink model to first 4226 iterations.


[I 2024-04-21 22:20:18,394] Trial 0 finished with value: 0.9589545086001503 and parameters: {'depth': 8, 'learning_rate': 0.19012957765207258, 'l2_leaf_reg': 0.3009774766532844, 'bagging_temperature': 0.07862959138237768, 'border_count': 160, 'random_strength': 61.88799479804526, 'min_data_in_leaf': 1}. Best is trial 0 with value: 0.9589545086001503.


0:	learn: 1.2010112	test: 1.2069814	best: 1.2069814 (0)	total: 74.7ms	remaining: 6m 13s
bestTest = 0.8981358073
bestIteration = 16
Shrink model to first 17 iterations.


[I 2024-04-21 22:20:28,751] Trial 1 finished with value: 0.8744578808129274 and parameters: {'depth': 6, 'learning_rate': 0.40519344070742164, 'l2_leaf_reg': 0.08990483952245525, 'bagging_temperature': 0.3583561220076431, 'border_count': 89, 'random_strength': 55.92917647100746, 'min_data_in_leaf': 15}. Best is trial 0 with value: 0.9589545086001503.


...

[I 2024-04-22 03:44:52,985] Trial 37 finished with value: 0.9655730230225981 and parameters: {'depth': 10, 'learning_rate': 0.17105062837714524, 'l2_leaf_reg': 5.770445248824186, 'bagging_temperature': 0.907832755354328, 'border_count': 174, 'random_strength': 60.219846868632494, 'min_data_in_leaf': 17}. Best is trial 33 with value: 0.9663749572145994.


0:	learn: 1.6442775	test: 1.6468396	best: 1.6468396 (0)	total: 259ms	remaining: 21m 34s
4999:	learn: 0.1946618	test: 0.5888861	best: 0.5888787 (4997)	total: 13m 21s	remaining: 0us
bestTest = 0.5888787109
bestIteration = 4997
Shrink model to first 4998 iterations.


[I 2024-04-22 03:58:27,076] Trial 38 finished with value: 0.9641045412588201 and parameters: {'depth': 9, 'learning_rate': 0.16675840964908029, 'l2_leaf_reg': 4.9193893138446905, 'bagging_temperature': 0.8701950569735756, 'border_count': 176, 'random_strength': 29.17881757032693, 'min_data_in_leaf': 17}. Best is trial 33 with value: 0.9663749572145994.


0:	learn: 1.2735668	test: 1.2776404	best: 1.2776404 (0)	total: 355ms	remaining: 29m 33s
bestTest = 0.5878391927
bestIteration = 2601
Shrink model to first 2602 iterations.


[I 2024-04-22 04:10:19,781] Trial 39 finished with value: 0.9638356220773353 and parameters: {'depth': 10, 'learning_rate': 0.2617675879399603, 'l2_leaf_reg': 3.9112810480603915, 'bagging_temperature': 0.8915867007503682, 'border_count': 194, 'random_strength': 58.862655482076725, 'min_data_in_leaf': 17}. Best is trial 33 with value: 0.9663749572145994.


In [6]:
print("Количество завершенных испытаний: {}".format(len(study.trials)))

trial = study.best_trial
print(f"Лучшее значение: {trial.value}\n")

print("Гиперпараметры:")
for key, value in trial.params.items():
    print("   {}: {}".format(key, value))

Количество завершенных испытаний: 40
Лучшее значение: 0.9663749572145994

Гиперпараметры:
   depth: 10
   learning_rate: 0.17202049904649439
   l2_leaf_reg: 7.606154289229104
   bagging_temperature: 0.8673588076824617
   border_count: 68
   random_strength: 61.14863444620232
   min_data_in_leaf: 14


## <b>4 <span style='color:#4682B4'>|</span> Обучение модели </b>

In [7]:
best_params = study.best_params  

model = CatBoostClassifier(iterations=10_000,
                           loss_function='MultiClass',
                           task_type='GPU',
                           **best_params)


model.fit(x_train,
          y_train,
          cat_features=cat_features,
          eval_set=(x_val, y_val),
          verbose=1000,
          early_stopping_rounds=100)

0:	learn: 1.6174049	test: 1.6199918	best: 1.6199918 (0)	total: 303ms	remaining: 50m 33s
1000:	learn: 0.4475923	test: 0.6746224	best: 0.6746224 (1000)	total: 3m 5s	remaining: 27m 48s
2000:	learn: 0.2888432	test: 0.6080876	best: 0.6080876 (2000)	total: 6m 32s	remaining: 26m 9s
3000:	learn: 0.2125650	test: 0.5809010	best: 0.5809010 (3000)	total: 10m 4s	remaining: 23m 30s
4000:	learn: 0.1714029	test: 0.5723939	best: 0.5723563 (3997)	total: 13m 36s	remaining: 20m 24s
bestTest = 0.5717523437
bestIteration = 4392
Shrink model to first 4393 iterations.


In [8]:
y_pred_proba = model.predict_proba(x_val)
weighted_roc_auc(y_val, y_pred_proba, model.classes_, weights_dict)

0.9662047594042886

## <b>5 <span style='color:#4682B4'>|</span> Submission </b>

In [9]:
sample_submission_df = pd.read_csv('/kaggle/input/alfa-x-finu-hack/sample_submission.csv')

last_m_test_df = test_df[test_df["date"] == "month_6"]
last_m_test_df = last_m_test_df.drop(['id', 'date'], axis=1)

test_pred_proba = model.predict_proba(last_m_test_df)
test_pred_proba_df = pd.DataFrame(test_pred_proba, columns=model.classes_)
test_pred_proba_df.head()

,{other},{},"{α, β}","{α, γ}","{α, δ}","{α, ε, η}","{α, ε, θ}","{α, ε, ψ}","{α, ε}","{α, η}","{α, θ}","{α, λ}","{α, μ}","{α, π}","{α, ψ}",{α},{λ}
0,0.000215,0.001067,9.286726e-04,0.004192,1.342531e-03,1.180768e-06,2.264495e-05,4.163266e-06,0.000424,0.000283,3.308486e-04,6.741641e-06,0.000011,2.068987e-07,0.000098,0.991073,2.028584e-07
1,0.000028,0.412001,7.896603e-07,0.000011,1.030863e-06,1.142840e-07,8.608535e-08,3.005169e-08,0.000004,0.000182,6.242754e-07,6.902805e-07,0.000002,4.016607e-09,0.000021,0.587745,1.743911e-06
2,0.001142,0.000687,3.131240e-04,0.005420,4.973045e-04,4.888271e-06,7.468658e-06,1.098751e-05,0.000136,0.000218,1.414612e-04,5.835006e-05,0.000008,3.026572e-07,0.000301,0.991054,4.667873e-07
3,0.011543,0.435397,1.997292e-05,0.000297,5.612789e-07,6.373468e-05,5.668058e-06,4.851931e-07,0.000005,0.226342,1.223628e-03,3.757970e-07,0.000148,7.759173e-08,0.000031,0.324922,2.805962e-07
4,0.003851,0.010537,5.868169e-03,0.008129,6.173457e-05,3.011136e-06,2.713406e-06,1.136141e-06,0.000122,0.043845,1.337683e-03,3.104938e-06,0.000118,5.638335e-07,0.000002,0.926116,6.051511e-07


In [10]:
sample_submission_df[model.classes_] = test_pred_proba_df
sample_submission_df.to_csv("sub.csv", index=False)